In [88]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
from __future__ import print_function, absolute_import

import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from pydrake.all import (
    DiagramBuilder,
    FloatingBaseType,
    RigidBodyTree,
    RigidBodyPlant,
    SignalLogger, 
    Simulator, 
    VectorSystem
)
from pydrake.systems.controllers import (
    DynamicProgrammingOptions, FittedValueIteration, PeriodicBoundaryCondition)
from pydrake.examples.acrobot import AcrobotPlant
from underactuated import (
    PlanarRigidBodyVisualizer
)



plant = AcrobotPlant()
simulator = Simulator(plant)
options = DynamicProgrammingOptions()

# State: (theta1, theta2, theta1_dot, theta2_dot) Input: Elbow torque

def min_time_cost(context):
    x = context.get_continuous_state_vector().CopyToVector()
    x[0] = x[0] - math.pi
    if x.dot(x) < .01: # seeks get x to (math.pi, 0., 0., 0.) # was 0.01
        return 0.
    return 1.


def quadratic_regulator_cost(context):
    x = context.get_continuous_state_vector().CopyToVector()
    x[0] = x[0] - math.pi
    u = plant.EvalVectorInput(context, 0).CopyToVector()
    
    if x.dot(x) < .1: # seeks get x to (math.pi, 0., 0., 0.) # was 0.01
        return 0.
#     return 0.1*x.dot(x) + 0.1*u.dot(u) + 1000
    return 1. + 2*x.dot(x)/(4*math.pi**2) + u.dot(u)/(300**2)


if (False):
    cost_function = min_time_cost
    input_limit = 100.
    options.convergence_tol = 0.001
    state_steps = 19
    input_steps = 19
else:
    cost_function = quadratic_regulator_cost
    input_limit = 300.
    options.convergence_tol = 0.001
    state_steps = 15
    input_steps = 15

####### SETTINGS #######
# State: (x, theta, x_dot, theta_dot)
# Previous Best... (quad. cost)
# qbins = np.linspace(0., 2. * math.pi, state_steps)
# qbins2 = np.linspace(0., 2. * math.pi, state_steps)
# qdotbins = np.linspace(-10., 10., state_steps)
# qdotbins2 = np.linspace(-10., 10., state_steps)
# options.periodic_boundary_conditions = [
#     PeriodicBoundaryCondition(0, 0., 2.*math.pi),
#     PeriodicBoundaryCondition(1, 0., 2.*math.pi),
# ]

# Test
# state_steps = 15
# input_steps = 21
# side_bins = 7
# mid_bins = 7
# assert (side_bins*2+mid_bins) % 2 == 1
# offset = 0.2
# qbins = np.hstack((np.linspace(0., math.pi-offset, side_bins), np.linspace(math.pi-offset, math.pi+offset, mid_bins), np.linspace(math.pi+offset, 2*math.pi, side_bins)))
# qbins2 = np.hstack((np.linspace(0., math.pi-offset, side_bins), np.linspace(math.pi-offset, math.pi+offset, mid_bins), np.linspace(math.pi+offset, 2*math.pi, side_bins)))
# qdotbins = np.linspace(-10., 10., state_steps)
# qdotbins2 = np.linspace(-10., 10., state_steps)
# options.periodic_boundary_conditions = [
#     PeriodicBoundaryCondition(0, 0., 2.*math.pi),
#     PeriodicBoundaryCondition(1, 0., 2.*math.pi),
# ]

# Test 2 - what will it actually take to stabilize the upright point?
# options.convergence_tol = 0.0005
# input_limit = 100.
# state_steps = 21
# input_steps = 21
# side_bins = 8
# mid_bins = 7
# assert (side_bins*2+mid_bins) % 2 == 1
# q_offset = 0.2
# qbins = np.linspace(math.pi-q_offset, math.pi+q_offset, mid_bins)
# qbins2 = np.linspace(math.pi-q_offset, math.pi+q_offset, mid_bins)
# qdotbins = np.linspace(-3., 3., state_steps)
# qdotbins2 = np.linspace(-3., 3., state_steps)
# options.periodic_boundary_conditions = [
#     PeriodicBoundaryCondition(0, 0., 2.*math.pi),
#     PeriodicBoundaryCondition(1, 0., 2.*math.pi),
# ]
input_grid = [set(np.linspace(-input_limit, input_limit, input_steps))]

# Test 3
state_steps = 19
input_steps = 19
side_bins = 9
mid_bins = 9
assert (side_bins*2+mid_bins) % 2 == 1
offset = 0.2
qbins = np.hstack((np.linspace(0., math.pi-offset, side_bins), np.linspace(math.pi-offset, math.pi+offset, mid_bins), np.linspace(math.pi+offset, 2*math.pi, side_bins)))
qbins2 = np.hstack((np.linspace(0., math.pi-offset, side_bins), np.linspace(math.pi-offset, math.pi+offset, mid_bins), np.linspace(math.pi+offset, 2*math.pi, side_bins)))
qdotbins = np.linspace(-10., 10., state_steps)
qdotbins2 = np.linspace(-10., 10., state_steps)
options.periodic_boundary_conditions = [
    PeriodicBoundaryCondition(0, 0., 2.*math.pi),
    PeriodicBoundaryCondition(1, 0., 2.*math.pi),
]
u_bins = np.hstack((np.linspace(-input_limit, -2, side_bins), np.linspace(-2, 2, mid_bins), np.linspace(2, input_limit, side_bins)))
input_grid = [set(u_bins)]

# Test log:
# 1) Trying to add stabilizing points to upright, also generally increasing input steps, and qbins steps, prob. will take a while!
# 2) Just trying to stabilize the top, took like overnight and no convergence...
# 3) Going back to test 1, and using a very similar cost function to cartpole, also with more bins.
# 4) Reduced 800 -> 300. WOW that helped a lot...
# 5) 300 -> 180...
# 6) 180 -> 300 and increased reward hole 0.01 -> 0.1

state_grid = [set(qbins), set(qbins2), set(qdotbins), set(qdotbins2)]
# input_grid = [set(np.linspace(-input_limit, input_limit, input_steps))]
timestep = 0.001


policy, cost_to_go = FittedValueIteration(simulator, cost_function,
                                          state_grid, input_grid,
                                          timestep, options)

In [57]:
from pydrake.all import (BarycentricMesh, BarycentricMeshSystem)
def save_policy(): # binds to policy and state_grid
    output_values = policy.get_output_values()
    np.save('numpy_saves/pi_b_mesh_init__acrobot', state_grid)
    np.save('numpy_saves/pi_output_values__acrobot', output_values)
def load_policy():
    b_mesh_init = np.load('numpy_saves/pi_b_mesh_init__acrobot.npy').tolist()
    output_values = np.load('numpy_saves/pi_output_values__acrobot.npy')
    b_mesh = BarycentricMesh(b_mesh_init)
    return BarycentricMeshSystem(b_mesh, output_values)
# save_policy()
test = load_policy()

In [58]:
# Animate the resulting policy.
builder = DiagramBuilder()
tree = RigidBodyTree("/opt/underactuated/src/acrobot/acrobot.urdf",
                     FloatingBaseType.kFixed)
plant = RigidBodyPlant(tree)
plant_system = builder.AddSystem(plant)


# TODO(russt): add wrap-around logic to barycentric mesh
# (so the policy has it, too)
class WrapTheta(VectorSystem):
    def __init__(self):
        VectorSystem.__init__(self, 4, 4)

    def _DoCalcVectorOutput(self, context, input, state, output):
        output[:] = input
        twoPI = 2.*math.pi
        output[0] = output[0] - twoPI * math.floor(output[0] / twoPI)
        output[1] = output[1] - twoPI * math.floor(output[1] / twoPI)


wrap = builder.AddSystem(WrapTheta())
builder.Connect(plant_system.get_output_port(0), wrap.get_input_port(0))
vi_policy = builder.AddSystem(test)
# vi_policy = builder.AddSystem(policy)
builder.Connect(wrap.get_output_port(0), vi_policy.get_input_port(0))
builder.Connect(vi_policy.get_output_port(0), plant_system.get_input_port(0))

logger = builder.AddSystem(SignalLogger(4))
logger._DeclarePeriodicPublish(0.033333, 0.0)
builder.Connect(plant_system.get_output_port(0), logger.get_input_port(0))

diagram = builder.Build()
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)

state = simulator.get_mutable_context().get_mutable_continuous_state_vector()
state.SetFromVector([1., 0.0, 0.0, 0.0])

# from traj.dircol import (
#     visualize_policy_system,
# )
# simulator, tree, logger = visualize_policy_system(policy, "acrobot")

In [59]:
simulator.StepTo(5.)

In [60]:
# Visualize the result as a video.
vis = PlanarRigidBodyVisualizer(tree, xlim=[-4., 4.], ylim=[-4., 4.])
ani = vis.animate(logger, repeat=True)

# plt.show()
# Things added to get visualizations in an ipynb
plt.close(vis.fig)
HTML(ani.to_html5_video())

## Let's verify the cost to go against cost of many random (w/ random inits) dircols!

## Let's evaluate the policy against (u, x) of many random (w/ random inits) dircols!

## Let's see what nets we can fit to this policy!

## Can a net fit to this policy when it only get's stochastic samples of the known optimal policy

## Can traj opt inited with the above fitted optimal policy stay in place / find the optimal off of a small perturbation?